<a href="https://colab.research.google.com/github/AndrewHolm/autoencoder-arduino-nano/blob/main/autoencoder_accel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Data from arduino (200 1 second samples at 50hz in 3 dimmensions)
# Pull side is a sharp left swipe with the arduino pointed at the screen (y), z least change
# Pull up is a sharp swipe up with arduino starting pointing at screen (y), x least change

# Load the CSV file into a DataFrame
df = pd.read_csv('accel_up_left_data.csv')

# Extract the moving and not moving data into separate arrays
pull_side = df['pull_side'].to_numpy()
pull_up = df['pull_up'].to_numpy()
print(pull_side.shape)

(30000,)


In [ ]:
from sklearn.preprocessing import StandardScaler

pull_up = pull_up.reshape(200, 50, 3)
pull_side = pull_side.reshape(200, 50, 3)


# Normalize the data (zero mean and unit variance)
# Reshape the data to have shape (4200*50, 3) for normalization

normalized_data_up = pull_up.reshape(-1, 3)
normalized_data_side = pull_side.reshape(-1, 3)

scaler = StandardScaler()

normalized_data_up = scaler.fit_transform(normalized_data_up)

normalized_data_side = scaler.fit_transform(normalized_data_side)

# Reshape the normalized data back to the original shape (400, 50, 3)
normalized_data_up = normalized_data_up.reshape(pull_up.shape)
normalized_data_side = normalized_data_side.reshape(pull_side.shape)

raw_data = np.concatenate((normalized_data_side, normalized_data_up), axis=0) # combine (400, 50, 3)

# Create labels for the data
num_samples, sample_rate, dim = raw_data.shape # num_samples = 400, sample_rate = 50

# create array for lables, 200 to represent side swipe (0), 200 to represent up swipe (1)
labels = np.concatenate((np.zeros(num_samples//2), np.ones(num_samples//2)))



# x_train -> data y_train -> labels, use 75% for training/ 25% for testing
X_train, X_test, y_train, y_test = train_test_split(raw_data, labels, test_size=0.25, random_state=42)
sample1 = raw_data.reshape(400,150)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np

normalized_data = sample1

input_dim = normalized_data.shape[1]
# compression size
latent_dim = 64

# Create the encoder model
encoder = Sequential()
encoder.add(Dense(128, activation='relu', input_shape=(input_dim,)))
encoder.add(Dense(latent_dim, activation='relu'))

# Create the decoder model
decoder = Sequential()
decoder.add(Dense(128, activation='relu', input_shape=(latent_dim,)))
decoder.add(Dense(input_dim, activation='linear'))

# Combine the encoder and decoder models into the autoencoder model
autoencoder = Sequential([encoder, decoder])

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

train_data, val_data = train_test_split(normalized_data, test_size=0.2, random_state=42)

# Training
autoencoder.fit(train_data, train_data, epochs=100, batch_size=32, shuffle=True, validation_data=(val_data, val_data))

# Compression and Decompression
encoded_data = encoder.predict(normalized_data)
reconstructed_data = decoder.predict(encoded_data)

Epoch 1/100
10/10 [==============================] - 1s 17ms/step - loss: 0.9692 - val_loss: 0.7690
Epoch 2/100
10/10 [==============================] - 0s 5ms/step - loss: 0.7599 - val_loss: 0.6060
Epoch 3/100
10/10 [==============================] - 0s 6ms/step - loss: 0.6107 - val_loss: 0.4975
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5100 - val_loss: 0.4319
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4404 - val_loss: 0.3888
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3970 - val_loss: 0.3598
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3606 - val_loss: 0.3411
Epoch 8/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3347 - val_loss: 0.3256
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3097 - val_loss: 0.3145
Epoch 10/100
10/10 [==============================] - 0s 4ms/step - loss: 0.2911 - val_loss: 0.3060
Epoch 11

In [ ]:
# convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(encoder)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# convert to a c header file
!apt-get update && apt-get -qq install xxd
!xxd -i model.tflite > encoder.h

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,012 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,059 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,351 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,779 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu

In [ ]:
# print(reconstructed_data[120])
# print(normalized_data[120])
print(encoded_data[0])
print(encoded_data.shape)

[0.31626737 0.         1.5823089  0.51153034 1.7711321  0.
 1.6641611  1.4744486  3.0693388  2.2044382  0.8072093  0.80888855
 2.5725346  0.5803729  1.4528929  0.49476996 0.26514265 3.0514514
 0.621678   0.93526906 0.         0.989805   1.5376823  0.
 0.         0.44638196 0.         0.         1.5941517  0.
 0.         0.5660197  0.         0.17064214 0.         0.
 1.2886528  0.52150565 0.68582064 1.0517743  1.8238121  1.7886131
 2.2726376  2.2980254  0.94821095 0.         1.1249874  1.6019126
 0.         1.4227327  0.96348584 0.         1.048302   2.4122872
 0.         1.6578296  2.1278427  0.         0.61143535 1.1489259
 4.2285404  0.         0.         2.678676  ]
(400, 64)


In [ ]:
# copy encoded data from arduino
enc_up1 = np.array([0.00, 0.00, 0.00, 0.00, 1.70, 3.17, 3.28, 4.24, 3.08, 2.38, 1.66, 3.51, 0.00, 0.71, 0.00, 0.00, 1.98, 2.47, 1.19, 0.00, 2.16, 2.25, 1.04, 0.00, 0.00, 0.04, 1.79, 0.00, 1.86, 0.70, 0.00, 0.13, 3.93, 1.65, 0.00, 0.91, 1.11, 0.14, 0.16, 1.81, 2.58, 2.29, 0.89, 0.00, 0.00, 0.00, 0.98, 1.72, 0.33, 0.00, 0.00, 2.63, 5.83, 0.58, 2.42, 0.00, 0.00, 0.00, 0.70, 1.52, 0.00, 0.00, 2.52, 2.45])
enc_up2 = np.array([0.00, 0.00, 0.00, 0.00, 2.08, 1.32, 4.29, 3.66, 4.14, 2.74, 1.66, 3.74, 0.00, 0.20, 0.00, 0.00, 1.80, 1.86, 1.40, 0.17, 3.67, 2.55, 0.80, 0.00, 0.00, 0.00, 1.44, 0.00, 1.61, 0.79, 0.00, 0.49, 4.12, 1.13, 0.00, 0.04, 1.39, 0.00, 0.54, 0.99, 3.91, 2.38, 0.65, 0.00, 0.27, 0.07, 0.64, 2.21, 0.00, 0.00, 0.00, 2.57, 6.06, 2.21, 1.83, 1.09, 0.00, 0.00, 1.30, 2.59, 1.53, 0.00, 2.47, 2.68])
enc_left1 = np.array([0.00, 0.00, 0.00, 0.39, 0.66, 1.82, 1.87, 3.13, 1.50, 3.58, 1.50, 3.89, 0.00, 0.90, 0.00, 0.84, 0.50, 2.84, 0.02, 1.62, 2.65, 2.54, 0.33, 0.00, 0.53, 1.77, 0.86, 0.25, 0.89, 0.00, 0.00, 1.21, 0.14, 3.13, 0.00, 0.00, 1.22, 1.09, 2.50, 1.86, 2.69, 3.49, 0.00, 0.43, 0.79, 0.00, 0.00, 2.42, 0.00, 0.00, 1.13, 3.48, 5.63, 3.21, 1.28, 0.59, 0.00, 1.58, 2.03, 0.45, 3.24, 0.00, 0.00, 2.78])
enc_left2 = np.array([0.00, 0.00, 0.00, 1.10, 1.34, 1.21, 2.80, 2.76, 2.58, 3.70, 2.07, 3.95, 1.36, 2.46, 0.00, 1.30, 1.36, 1.47, 1.11, 0.46, 0.57, 1.09, 0.92, 1.01, 0.09, 4.16, 0.28, 0.00, 0.00, 1.43, 0.00, 0.91, 1.22, 2.87, 0.00, 0.00, 0.50, 1.35, 1.45, 2.89, 0.35, 2.70, 0.00, 0.09, 0.83, 0.00, 0.00, 1.08, 0.00, 0.58, 1.87, 0.00, 3.37, 0.75, 0.52, 0.00, 0.19, 0.00, 1.07, 1.24, 2.00, 0.00, 0.00, 2.00])

In [ ]:
# testing the reconstructed data (decoding data encoded on arduino)
recon_up1 = decoder.predict(np.expand_dims(enc_up1, axis=0))
recon_up2 = decoder.predict(np.expand_dims(enc_up2, axis=0))
recon_left1 = decoder.predict(np.expand_dims(enc_left1, axis=0))
recon_left2 = decoder.predict(np.expand_dims(enc_left2, axis=0))

# recon_up2 = decoder.predict(enc_up2)
# recon_left1 = decoder.predict(enc_left1)
# recon_left2 = decoder.predict(enc_left2)

1/1 [==============================] - 0s 25ms/step


In [ ]:
print(recon_up1)
print(recon_left1)

[[-1.1777471  -0.7864548  -0.00423748 -0.8340096  -0.06436472  0.34946492
  -1.1564169  -0.05503287  0.9681298  -0.79453564 -0.04033934  0.43210274
  -0.85697013 -0.73426     0.1606679  -0.740796   -0.8960267   0.45226198
  -0.8975558  -0.65448135  0.43582588 -0.81091267 -0.62177163  0.11555614
  -0.5283816  -0.46913233  0.17871419 -0.4469193  -0.06338474  0.5099238
  -1.0789574  -0.32392624  1.1360737  -1.139829   -0.6882303   0.59058195
  -1.3187705  -0.8289567   0.18980551 -0.9374849  -0.9057495  -0.01976958
  -0.86100405 -0.32776693  0.80619764 -0.78914535 -0.17817602  1.8191425
  -0.80221725 -0.39467627  0.8627856  -0.9274314  -1.9319025   0.38428134
  -0.98266745 -1.1330926   1.5092047  -1.0848328  -0.28474286  1.4670451
  -1.1392585  -0.23618406  1.4188731  -1.2222731  -0.8258652   0.9797459
  -1.1400794  -0.7946047   1.1580204  -1.058196   -0.55253184  1.133884
  -0.97304857 -0.91278315  0.7961787  -0.905085   -1.257393    0.9198602
  -0.6960232  -1.0807447   1.0068597  -0.8782